# Part 2: SQLite + Python

In [1]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def select_all_tasks(conn):
    cur = conn.cursor()
    query1 = """
        SELECT *
        FROM FACILITIES
        """    
    cur.execute(query1)
    rows = cur.fetchall()
    for row in rows:
        print(row)
        
def main():
    database = 'sqlite_db_pythonsqlite.db'
    conn = create_connection(database)
    with conn:
        print('2. Query all tasks')
        select_all_tasks(conn)

if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd

conn = sqlite3.connect("./sqlite_db_pythonsqlite.db")
cur = conn.cursor()

### Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [3]:
produce_list='''
SELECT facility_name, total_revenue from (
    SELECT f.name as facility_name,
    sum(CASE WHEN b.memid = 0 THEN f.guestcost*b.slots
        ELSE f.membercost*b.slots
        END) AS total_revenue
    FROM Facilities as f
    INNER JOIN Bookings as b
    USING (facid)
    GROUP BY f.name
    ) AS sample
WHERE total_revenue > 1000
ORDER BY total_revenue;
'''

pd.read_sql(produce_list,conn)

,facility_name,total_revenue
0,Badminton Court,1906.5
1,Squash Court,13468.0
2,Tennis Court 1,13860.0
3,Tennis Court 2,14310.0
4,Massage Room 2,14454.6
5,Massage Room 1,50351.6


### Produce a report of members and who recommended them in alphabetic, surname, firstname order

In [4]:
produce_report='''
SELECT m.memid, m.recommendedby, m.firstname, m.surname, r.firstname AS rec_firstname, r.surname AS rec_surname
FROM Members AS m
LEFT OUTER JOIN Members as r
ON r.memid = m.recommendedby
ORDER BY m.firstname, m.surname;
'''

pd.read_sql(produce_report,conn)

,memid,recommendedby,firstname,surname,rec_firstname,rec_surname
0,21,1,Anna,Mackenzie,Darren,Smith
1,12,9,Anne,Baker,Ponder,Stibbons
2,6,,Burton,Tracy,None,None
3,10,1,Charles,Owen,Darren,Smith
4,1,,Darren,Smith,None,None
5,37,,Darren,Smith,None,None
6,28,,David,Farrell,None,None
7,11,4,David,Jones,Janice,Joplette
8,17,13,David,Pinker,Jemima,Farrell
9,26,11,Douglas,Jones,David,Jones


### Find the facilities with their usage by member, but not guests

In [5]:
find_facilities = '''
SELECT facility_name, member_name, ROUND(sum_slots,1) AS monthly_usage from (
    SELECT f.name AS facility_name, 
    m.firstname || m.surname AS member_name,
    SUM(slots) AS sum_slots
    FROM Members AS m
    INNER JOIN Bookings as b
    USING (memid)
    INNER JOIN Facilities as f
    USING (facid)
    WHERE b.memid != 0
    GROUP BY f.facid, member_name
    ) as sub_q1
ORDER BY facility_name, member_name DESC
'''

pd.read_sql(find_facilities,conn)

,facility_name,member_name,monthly_usage
0,Badminton Court,TracySmith,102.0
1,Badminton Court,TimothyBaker,21.0
2,Badminton Court,TimRownam,12.0
3,Badminton Court,TimBoothe,36.0
4,Badminton Court,RamnareshSarwin,21.0
...,...,...,...
197,Tennis Court 2,DavidFarrell,3.0
198,Tennis Court 2,DarrenSmith,57.0
199,Tennis Court 2,CharlesOwen,141.0
200,Tennis Court 2,BurtonTracy,9.0


### Find the facilities usage by month, but not guests

In [6]:
find_facilities2='''
SELECT strftime('%m', starttime) AS month_2012, f.name AS facility_name, COUNT(*) AS monthly_usage
FROM Bookings AS b
    INNER JOIN Facilities AS f
    USING (facid)
    WHERE b.memid != 0
    GROUP BY month_2012, name
'''

pd.read_sql(find_facilities2,conn)

,month_2012,facility_name,monthly_usage
0,07,Badminton Court,51
1,07,Massage Room 1,77
2,07,Massage Room 2,4
3,07,Pool Table,103
4,07,Snooker Table,68
5,07,Squash Court,23
6,07,Table Tennis,48
7,07,Tennis Court 1,65
8,07,Tennis Court 2,41
9,08,Badminton Court,132
